In [26]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import warnings
import datetime
warnings.filterwarnings(action='ignore')

In [89]:
# ASOS 관측 데이터
weather = pd.read_csv('../data/기상/OBS_ASOS_DD_20210601231517.csv', encoding = 'cp949')
# ASOS 관측소 메타 데이터
coor = pd.read_csv('../data/기상/META_관측지점정보_20210603224446.csv',encoding='cp949')
# 산사태 발생 이력 데이터
land = pd.read_csv('../data/산사태 발생이력.csv', encoding = 'cp949')
# 행정동 경계 데이터
boundary = gpd.read_file('../data/행정동 경계/umd.shp',encoding='utf-8')

In [90]:
# 기상청 메타 데이터를 geopandas가 인식할 수 있도록 point 형태로 변환
coor['geometry'] = gpd.points_from_xy(coor.경도,coor.위도)
tmp = gpd.GeoDataFrame({'geometry':coor['geometry']})

# 기상청 페이지에 기재된대로 좌표계 형식 설정 
tmp.crs = {'init':'epsg:4326'}

# 행정동 경계 데이터도 같은 좌표계로 설정해준다.
boundary = boundary.to_crs({'init':'epsg:4326'})

In [91]:
# 산사태가 발생한 지역의 polygon데이터만 수집
land_bound = pd.DataFrame()
for i in set(land['umd']):
    land_bound = pd.concat([land_bound,boundary[boundary['ADM_DR_NM']==i.strip(' ')]],axis=0)
land_bound = land_bound.reset_index().drop('index',axis=1)

In [93]:
# 산사태가 발생한 지역에 속한 ASOS 관측소 찾기
coor['land'] = 0
land_bound['weather'] = 0
for i in range(len(coor)):
    for j in range(len(land_bound)):
        if land_bound['geometry'].iloc[j].contains(coor['geometry'].iloc[i]):
            print(coor['지점명'].iloc[i], land_bound['ADM_DR_NM'].iloc[j])
            coor['land'].iloc[i] = 1
            land_bound['weather'].iloc[j] = 1

울릉도 울릉읍
울진 울진읍
울진 울진읍
통영 정량동
진주 판문동
진주 초장동
북창원 중앙동
창원시 중앙동
의령군 의령읍
함양군 함양읍
영덕 영해면
경주시 선도동
거창 거창읍
거창 거창읍
합천 합천읍
산청 산청읍
남해 이동면


ASOS 관측소가 행정구역에 모두 속할 줄 알았는데 그렇지 않았다.  
거리를 계산해서 가까운 지역과 관측소 매칭

교체된 관측기에 대한 정보를 정제하고 다시 매칭

In [94]:
# 종료일이 존재하는 관측소를 검색
coor['종료일'] = coor['종료일'].astype('str')
coor[coor['종료일']!='nan']

,지점,시작일,종료일,지점명,지점주소,관리관서,위도,경도,노장해발고도(m),기압계(관측장비지상높이(m)),기온계(관측장비지상높이(m)),풍속계(관측장비지상높이(m)),강우계(관측장비지상높이(m)),geometry,land
5,99,2001-12-07,2013-10-22,문산,경기도 파주시문산읍 마정로46-29(파주기상대),파주기상대(99),37.8859,126.7665,29.42,31.40,1.70,10.0,0.5,POINT (126.76650 37.88590),0
7,100,1971-07-15,2006-11-06,대관령,강원도 평창군대관령면 경강로5372 대관령자동기상관측소,NaN,37.6869,128.7587,842.52,843.96,1.50,10.0,1.6,POINT (128.75870 37.68690),0
10,102,2000-11-01,2018-05-01,백령도,인천광역시 옹진군백령면 진촌리1031,백령도기상대(102),37.9661,124.6305,146.00,147.20,1.80,9.0,1.2,POINT (124.63050 37.96610),0
15,108,1907-10-01,2010-08-15,서울,서울특별시 종로구송월길 52서울기상관측소,서울기상관측소(108),37.5714,126.9658,85.50,86.50,1.50,10.0,0.6,POINT (126.96580 37.57140),0
17,112,1904-08-29,2013-10-22,인천,인천광역시 중구전동 25번지인천기상대,인천기상대(112),37.4776,126.6244,68.15,70.15,1.50,10.0,0.6,POINT (126.62440 37.47760),0
20,116,1984-01-01,1990-12-31,관악산,경기도 과천시중앙로 관악산길(관악산기상레이더관측소),NaN,37.4442,126.9639,626.76,627.96,1.40,3.9,1.1,POINT (126.96390 37.44420),0
22,119,1964-01-01,2019-07-24,수원,경기도 수원시권선구 권선로276,수도권기상청(119),37.2723,126.9853,34.84,35.84,1.60,18.7,1.1,POINT (126.98530 37.27230),0
27,130,1971-01-12,2020-12-04,울진,경상북도 울진군울진읍 현내항길157 울진지역기상서비스센터,대구(구 143),36.9918,129.4128,48.98,50.18,1.75,10.0,1.3,POINT (129.41280 36.99180),1
32,136,1973-01-01,1978-05-01,안동,경상북도 안동시열루재1길 16(운안동433-1) 안동기상대,NaN,36.5730,128.7073,139.39,141.40,1.50,10.0,0.6,POINT (128.70730 36.57300),0
36,140,1968-01-01,2003-12-01,군산,전라북도 군산시내흥동 425-10호군산지역기상서비스센터,NaN,35.9930,126.7057,25.57,30.74,1.50,18.0,0.6,POINT (126.70570 35.99300),0


산사태 발생 이력 데이터의 시작이 2011년 7월 9일  
이전에 관측을 중단한 관측소는 데이터에서 제외한다  

In [95]:
coor = coor.drop([4,8,9,12,15,32],axis=0)

In [96]:
# 데이터 정제 후 다시 매칭
# 모든 관측소와 거리를 계산하여 가장 가까운 관측소를 매칭
land_bound['ASOS'] = 0
for i in range(len(land_bound)):
    dis = []
    for j in range(len(coor)):
        dis.append(land_bound['geometry'].iloc[i].distance(coor['geometry'].iloc[j]))
    land_bound['ASOS'].iloc[i] = coor['지점명'].iloc[dis.index(min(dis))]

In [97]:
# 하나의 관측소가 커버하는 지역의 개수
for i in set(land_bound['ASOS']):
    print(i,len(land_bound[land_bound['ASOS']==i]))

울산 2
영덕 12
포항 19
밀양 18
구미 12
의령군 4
양산시 7
추풍령 11
창원 2
진주 24
경주시 6
광양시 6
안동 2
합천 23
장수 1
함양군 10
창원시 1
김해시 7
청송군 4
문경 1
거창 14
통영 12
상주 4
대구 3
영주 1
영천 5
울진 8
산청 11
봉화 1
북창원 7
마산 10
남해 9
울릉도 3


In [98]:
# 산사태 발생 지역에 데이터 프레임 하나씩

In [102]:
land_dict = {k.strip(' '):[] for k in land['umd']}
# 다음 지역에 대해선 예외처리한 후 직접 데이터프레임 병합
except_land = []
for i in land_dict.keys():
    if len(land_bound[land_bound['ADM_DR_NM']==i]) > 1:
        print(i,list(land_bound[land_bound['ADM_DR_NM']==i]['ASOS']))
        except_land.append(i)

상동면 ['김해시', '밀양']
성산면 ['합천', '밀양']
묘산면 ['합천', '합천']
병곡면 ['영덕', '함양군']
대가면 ['구미', '진주']
남면 ['구미', '남해', '구미', '남해']
서면 ['영천', '울릉도', '남해']
북면 ['울진', '울릉도', '북창원']
봉산면 ['추풍령', '합천']
덕곡면 ['합천', '합천']
동해면 ['포항', '마산']
자산동 ['추풍령', '마산']
북부동 ['대구', '김해시']
도산면 ['안동', '통영']
중앙동 ['포항', '영천', '대구', '진주', '통영', '양산시', '북창원']


In [103]:
land['date'] = land['date'].apply(lambda X:datetime.datetime(int(str(X)[:4]),int(str(X)[4:6]),int(str(X)[6:])))
weather['일시'] = pd.to_datetime(weather['일시'])

In [118]:
for i in land_dict.keys():
    if i in except_land:
        pass
    else:
        asos = land_bound[land_bound['ADM_DR_NM']==i]['ASOS'].iloc[0]
        asosDF = weather[weather['지점명']==asos].sort_values('일시')
        landDF = land[land['umd']==i]
        land_dict[i] = pd.merge(landDF,asosDF, left_on='date', right_on='일시', how='outer')
        land_dict[i]['sd'] = land_dict[i]['sd'].iloc[0]
        land_dict[i]['sgg'] = land_dict[i]['sgg'].iloc[0]
        land_dict[i]['umd'] = land_dict[i]['umd'].iloc[0]
        land_dict[i]['sum_cnt'] = land_dict[i]['sum_cnt'].fillna(0)
        land_dict[i]['sum_hpa'] = land_dict[i]['sum_hpa'].fillna(0)        

In [123]:
land_dict['내일동'].corr()

,sum_cnt,sum_hpa,지점,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
sum_cnt,1.000000,1.000000,NaN,0.016350,0.021497,0.023375,0.011015,-0.049499,NaN,0.095370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sum_hpa,1.000000,1.000000,NaN,0.016350,0.021497,0.023375,0.011015,-0.049499,NaN,0.095370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
지점,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
평균기온(°C),0.016350,0.016350,NaN,1.000000,0.978159,-0.092738,0.970665,-0.038737,NaN,0.198603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
최저기온(°C),0.021497,0.021497,NaN,0.978159,1.000000,-0.036991,0.908852,-0.087234,NaN,0.235056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5.0m 지중온도(°C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
합계 대형증발량(mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
합계 소형증발량(mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9-9강수(mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


산사태 발생 지역과 ASOS관측소를 거리 기준으로 매칭하여 데이터 셋을 구축하였다.  
다음 작업은 기상 데이터의 columns을 살펴보고 학습에 사용할 변수를 선택하는 작업이다.  